In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import os
os.environ["HF_DATASETS_CACHE"] = "/shared/3/projects/bangzhao/.hf_cache"
from datasets import load_dataset

/opt/anaconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torch.optim import AdamW
import torch.nn as nn

from transformers import *
from tokenizers import *

from bertPhoneme import BertEmbeddingsV2, BertModelV2, BertForMaskedLMV2, BertConfigV2, MaskedLMWithProsodyOutput

2025-07-22 17:55:30.442207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753206930.457336 3178664 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753206930.462059 3178664 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753206930.476701 3178664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753206930.476712 3178664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753206930.476713 3178664 computation_placer.cc:177] computation placer alr

In [5]:
import json
import random
from datasets import load_dataset

In [5]:
file_path = "/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl"
phonemes = set()

with open(file_path, 'r') as f:
    for i, line in enumerate(f):
        if i >= 5000:
            break
        phonemes.update(set(json.loads(line)['phoneme']))
    print(len(phonemes))

70


In [6]:
phonemes.update(['UNK', 'SIL'])

In [7]:
phoneme_vocab = {p: i for i, p in enumerate(phonemes)}

In [8]:
print(phoneme_vocab)

{'NG': 0, 'UH0': 1, 'S': 2, 'EY2': 3, 'EY1': 4, 'AE1': 5, 'OY1': 6, 'ER1': 7, 'SIL': 8, 'W': 9, 'JH': 10, 'UNK': 11, 'AA0': 12, 'B': 13, 'SH': 14, 'DH': 15, 'EH2': 16, 'AW2': 17, 'UH2': 18, 'AH0': 19, 'AH2': 20, 'AO2': 21, 'IY2': 22, 'IH0': 23, 'EY0': 24, 'OY2': 25, 'IH1': 26, 'Z': 27, 'TH': 28, 'AY2': 29, 'T': 30, 'OW0': 31, 'OY0': 32, 'spn': 33, 'P': 34, 'HH': 35, 'OW2': 36, 'K': 37, 'AE0': 38, 'OW1': 39, 'G': 40, 'M': 41, 'R': 42, 'N': 43, 'UW2': 44, 'AY0': 45, 'ER0': 46, 'AE2': 47, 'AA1': 48, 'F': 49, 'ZH': 50, 'UW1': 51, 'Y': 52, 'AW1': 53, 'D': 54, 'AY1': 55, 'L': 56, 'EH0': 57, 'AA2': 58, 'AW0': 59, 'AO0': 60, 'IY1': 61, 'UW0': 62, 'EH1': 63, 'ER2': 64, 'UH1': 65, 'IY0': 66, 'AH1': 67, 'AO1': 68, 'CH': 69, 'IH2': 70, 'V': 71}


In [9]:
phoneme_vocab_size = len(phoneme_vocab)
mask_token_id = phoneme_vocab["SIL"]
pad_token_id = 72
pad_cluster_id = 201
mask_prosody_id = 200


class HuggingFacePhonemeDataset(Dataset):
    def __init__(self, hf_dataset, vocab, mask_prob=0.15, max_length=512):
        self.dataset = hf_dataset
        self.vocab = vocab
        self.mask_prob = mask_prob
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        phonemes = sample["phoneme"]
        prosody_ids = sample["prosody_id_200"]

        input_ids = [self.vocab.get(p, self.vocab["UNK"]) for p in phonemes][:self.max_length]
        prosody_ids = prosody_ids[:self.max_length]

        labels = input_ids.copy()
        prosody_labels = prosody_ids.copy()
        for i in range(len(input_ids)):
            if random.random() < self.mask_prob:
                labels[i] = input_ids[i]
                input_ids[i] = self.vocab["SIL"]
                prosody_labels[i] = prosody_ids[i]
                prosody_ids[i] = 200
            else:
                labels[i] = -100
                prosody_labels[i] = -100

        pad_length = self.max_length - len(input_ids)
        input_ids.extend([pad_token_id] * pad_length)
        labels.extend([-100] * pad_length)
        prosody_ids.extend([pad_cluster_id] * pad_length)
        prosody_labels.extend([-100] * pad_length)
        attention_mask = [1] * (self.max_length - pad_length) + [0] * pad_length
        
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
            "prosody_ids": torch.tensor(prosody_ids, dtype=torch.long),
            "prosody_labels": torch.tensor(prosody_labels, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        }

In [10]:
hf_dataset = load_dataset("json", data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl", split="train")
hf_dataset = hf_dataset.select(range(49000))

Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

In [11]:
train_dataset = HuggingFacePhonemeDataset(hf_dataset, phoneme_vocab)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [15]:
vocab_size = phoneme_vocab_size
phoneme_vocab_size = len(phoneme_vocab)
max_length = 512


model_config = BertConfigV2(
    vocab_size=vocab_size+1,
    pad_token_id=pad_token_id,
    pad_cluster_id=201,
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=2,
    intermediate_size=512,
    max_position_embeddings=max_length,
    prosody_cluster_size=200 + 2
)

model = BertForMaskedLMV2(config=model_config)

# BERT-Base	768	12	12	3072
# BERT-Small 512	4	8	2048
# BERT-Mini	256	4	4	1024
# BERT-Tiny	128	2	2	512

In [16]:
model

BertForMaskedLMV2(
  (bert): BertModelV2(
    (embeddings): BertEmbeddingsV2(
      (word_embeddings): Embedding(73, 128, padding_idx=72)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (prosody_embeddings): Embedding(202, 128, padding_idx=201)
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

mlm_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
prosody_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

num_epochs = 2
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    print(f"\n Epoch {epoch + 1} starting...\n")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        prosody_ids = batch["prosody_ids"].to( device)
        prosody_labels = batch["prosody_labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids)

        mlm_loss = mlm_loss_fn(outputs.logits.view(-1, model.config.vocab_size), labels.view(-1))
        prosody_loss = prosody_loss_fn(outputs.prosody_logits.view(-1, model.config.prosody_cluster_size), prosody_labels.view(-1))

        total_batch_loss = mlm_loss + prosody_loss
        total_batch_loss.backward()
        optimizer.step()

        total_loss += total_batch_loss.item()

        # Report every 100 steps
        if (step + 1) % 100 == 0:
            avg_loss = total_loss / (step + 1)
            print(f"  Step {step + 1:>4} | Avg Loss: {avg_loss:.4f}")

    avg_epoch_loss = total_loss / len(train_loader)
    print(f"\n Epoch {epoch + 1} completed. Avg Loss: {avg_epoch_loss:.4f}")



 Epoch 1 starting...

  Step  100 | Avg Loss: 7.7207
  Step  200 | Avg Loss: 7.4372
  Step  300 | Avg Loss: 7.2869
  Step  400 | Avg Loss: 7.1877
  Step  500 | Avg Loss: 7.1178
  Step  600 | Avg Loss: 7.0637
  Step  700 | Avg Loss: 7.0214
  Step  800 | Avg Loss: 6.9855
  Step  900 | Avg Loss: 6.9540
  Step 1000 | Avg Loss: 6.9291
  Step 1100 | Avg Loss: 6.9061
  Step 1200 | Avg Loss: 6.8867
  Step 1300 | Avg Loss: 6.8677
  Step 1400 | Avg Loss: 6.8510
  Step 1500 | Avg Loss: 6.8356

 Epoch 1 completed. Avg Loss: 6.8310

 Epoch 2 starting...

  Step  100 | Avg Loss: 6.6188
  Step  200 | Avg Loss: 6.6098
  Step  300 | Avg Loss: 6.6056
  Step  400 | Avg Loss: 6.6009
  Step  500 | Avg Loss: 6.5975
  Step  600 | Avg Loss: 6.5922
  Step  700 | Avg Loss: 6.5895
  Step  800 | Avg Loss: 6.5863
  Step  900 | Avg Loss: 6.5836
  Step 1000 | Avg Loss: 6.5795
  Step 1100 | Avg Loss: 6.5759
  Step 1200 | Avg Loss: 6.5728
  Step 1300 | Avg Loss: 6.5692
  Step 1400 | Avg Loss: 6.5654
  Step 1500 | Avg

In [21]:
# hf_dataset_full = load_dataset(
#     "json",
#     data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl",
#     split="train"
# )

hf_dataset = load_dataset("json", data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data_6features/output_part_1_20kSample.jsonl", split="train")
# hf_test_dataset = hf_dataset_full.select(range(49000, 50000))
hf_dataset_test = hf_dataset.select(range(19900, 20000))

# === Step 2: Wrap the dataset with HuggingFacePhonemeDataset ===
# test_dataset = HuggingFacePhonemeDataset(
#     hf_dataset=hf_test_dataset,
#     vocab=phoneme_vocab,
#     mask_prob=0.15,
#     max_length=512
# )

test_dataset = HuggingFacePhonemeDataset(hf_dataset_test, phoneme_vocab)

Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

In [22]:
sample = test_dataset[2]

input_ids = sample["input_ids"].unsqueeze(0).to(model.device)
labels = sample["labels"].unsqueeze(0).to(model.device)
prosody_ids = sample["prosody_ids"].unsqueeze(0).to(model.device)
prosody_labels = sample["prosody_labels"].unsqueeze(0).to(model.device)
attention_mask = sample["attention_mask"].unsqueeze(0).to(model.device)

In [23]:
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, prosody_ids=prosody_ids, attention_mask=attention_mask)

pred_phonemes = torch.argmax(outputs.logits, dim=-1)[0]         # [seq_len]
pred_prosody = torch.argmax(outputs.prosody_logits, dim=-1)[0]  # [seq_len]

# Evaluation
id2phoneme = {v: k for k, v in phoneme_vocab.items()}
mask_positions = (labels != -100).nonzero(as_tuple=True)[1].tolist()

results = []
correct_phoneme = 0
correct_prosody = 0

for i in mask_positions:
    gt_ph_id = labels[0, i].item()
    pred_ph_id = pred_phonemes[i].item()

    gt_pr_id = prosody_labels[0, i].item()
    pred_pr_id = pred_prosody[i].item()

    phoneme_correct = gt_ph_id == pred_ph_id
    prosody_correct = gt_pr_id == pred_pr_id

    if phoneme_correct:
        correct_phoneme += 1
    if prosody_correct:
        correct_prosody += 1

    results.append({
        "position": i,
        "phoneme_correct": phoneme_correct,
        "phoneme_gt": id2phoneme.get(gt_ph_id, "UNK"),
        "phoneme_pred": id2phoneme.get(pred_ph_id, "UNK"),
        "phoneme_gt_id": gt_ph_id,
        "phoneme_pred_id": pred_ph_id,
        "prosody_correct": prosody_correct,
        "prosody_gt_id": gt_pr_id,
        "prosody_pred_id": pred_pr_id,
    })

# Accuracy
total = len(mask_positions)
phoneme_accuracy = correct_phoneme / total if total > 0 else None
prosody_accuracy = correct_prosody / total if total > 0 else None

print(f"\nPhoneme Accuracy: {phoneme_accuracy:.2%}" if phoneme_accuracy is not None else "No masked phoneme positions.")
print(f"Prosody Accuracy: {prosody_accuracy:.2%}" if prosody_accuracy is not None else "No masked prosody positions.")

for r in results:
    print(f"Pos {r['position']:>3} | Phoneme: Pred={r['phoneme_pred']} GT={r['phoneme_gt']} → {'True' if r['phoneme_correct'] else 'False'}")
    print(f" | Prosody: Pred={r['prosody_pred_id']} GT={r['prosody_gt_id']} → {'True' if r['prosody_correct'] else 'False'}")



Phoneme Accuracy: 40.28%
Prosody Accuracy: 8.33%
Pos   5 | Phoneme: Pred=AH0 GT=IH1 → False
 | Prosody: Pred=153 GT=50 → False
Pos   6 | Phoneme: Pred=N GT=S → False
 | Prosody: Pred=153 GT=133 → False
Pos   9 | Phoneme: Pred=G GT=G → True
 | Prosody: Pred=7 GT=113 → False
Pos  16 | Phoneme: Pred=AH0 GT=M → False
 | Prosody: Pred=7 GT=100 → False
Pos  18 | Phoneme: Pred=ER0 GT=AA1 → False
 | Prosody: Pred=7 GT=71 → False
Pos  26 | Phoneme: Pred=AY1 GT=IH1 → False
 | Prosody: Pred=153 GT=198 → False
Pos  31 | Phoneme: Pred=R GT=R → True
 | Prosody: Pred=153 GT=44 → False
Pos  44 | Phoneme: Pred=UW1 GT=UW1 → True
 | Prosody: Pred=7 GT=24 → False
Pos  61 | Phoneme: Pred=DH GT=DH → True
 | Prosody: Pred=153 GT=47 → False
Pos  63 | Phoneme: Pred=W GT=W → True
 | Prosody: Pred=7 GT=118 → False
Pos  65 | Phoneme: Pred=Z GT=K → False
 | Prosody: Pred=7 GT=23 → False
Pos  72 | Phoneme: Pred=AH0 GT=Y → False
 | Prosody: Pred=7 GT=184 → False
Pos  73 | Phoneme: Pred=IH1 GT=EH1 → False
 | Prosody

In [24]:
from tqdm import tqdm

total_phoneme_correct = 0
total_prosody_correct = 0
total_masked_positions = 0

model.eval()

for idx in tqdm(range(len(test_dataset)), desc="Evaluating samples"):
    sample = test_dataset[idx]

    input_ids = sample["input_ids"].unsqueeze(0).to(model.device)
    labels = sample["prosody_labels"].unsqueeze(0).to(model.device)
    prosody_ids = sample["prosody_ids"].unsqueeze(0).to(model.device)
    prosody_labels = sample["prosody_labels"].unsqueeze(0).to(model.device)
    attention_mask = sample["attention_mask"].unsqueeze(0).to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids, attention_mask=attention_mask)

    pred_phonemes = torch.argmax(outputs.logits, dim=-1)[0]
    pred_prosody = torch.argmax(outputs.prosody_logits, dim=-1)[0]
    mask_positions = (labels != -100).nonzero(as_tuple=True)[1].tolist()

    for i in mask_positions:
        gt_ph_id = labels[0, i].item()
        pred_ph_id = pred_phonemes[i].item()
        gt_pr_id = prosody_labels[0, i].item()
        pred_pr_id = pred_prosody[i].item()

        total_masked_positions += 1
        if gt_ph_id == pred_ph_id:
            total_phoneme_correct += 1
        if gt_pr_id == pred_pr_id:
            total_prosody_correct += 1

# === Final accuracy calculation ===
if total_masked_positions > 0:
    avg_phoneme_acc = total_phoneme_correct / total_masked_positions
    avg_prosody_acc = total_prosody_correct / total_masked_positions
else:
    avg_phoneme_acc = avg_prosody_acc = None

print(f"\n✅ Evaluated {total_masked_positions} masked positions over {len(test_dataset)} samples")
print(f"Phoneme Accuracy: {avg_phoneme_acc:.2%}" if avg_phoneme_acc is not None else "No masked phoneme positions.")
print(f"Prosody Accuracy: {avg_prosody_acc:.2%}" if avg_prosody_acc is not None else "No masked prosody positions.")

Evaluating samples: 100%|████████████████████| 1000/1000 [00:24<00:00, 40.70it/s]


✅ Evaluated 74471 masked positions over 1000 samples
Phoneme Accuracy: 0.42%
Prosody Accuracy: 11.27%
